In [ ]:
%load_ext autotime
import os, sys
import pandas as pd
pd.set_option('display.max_colwidth', 500)
sys.path.append('/root/car_sales_QA/src')

from retriever import Retriever
from LLM_utils import LLM
from prompt import generate_prompt

# import prompt
# import importlib
# importlib.reload(prompt)

# from prompt import generate_prompt

In [ ]:
import importlib
import prompt
importlib.reload(prompt)
from prompt import generate_prompt

In [ ]:
retriever = Retriever(embedder = 'BAAI/bge-m3',reranker = 'BAAI/bge-reranker-v2-m3',collection_name = 'sales_qa')
model_id = 'Qwen/Qwen2.5-3B-Instruct'
rewriter = LLM(model_id)

model_id = 'ep-20250213200344-crq6r'
gen_llm = LLM(model_id,api_key = os.getenv('ARK_API_KEY'),base_url = os.getenv('ARK_BASE_URL'),stream = True)

In [ ]:
history = []

In [ ]:
query = """I want to compare the BMW cars with the porsche cars you mentioned the first time"""
rewrite_prompt = generate_prompt('rewrite', query, history, reranked_chunks=None)
rewritten_query = rewriter(rewrite_prompt)

recall_docs, reranked_chunks = retriever.retrieve(rewritten_query)

In [ ]:
generation_prompt = generate_prompt('generate', query, history, reranked_chunks)
#generator = generation_llm(gen_llm,generation_prompt)
generator = gen_llm(generation_prompt)
response = ''
for token in generator:
    response += token
    print(token, end = "")
history.append({'User':rewritten_query, "Assistant": response})

In [ ]:
history[-1]

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()
model = ChatCompletionsClient(
    endpoint=os.getenv('AZURE_ENDPOINT'),
    credential=AzureKeyCredential(os.getenv('AZURE_API_KEY')),
)

from azure.ai.inference.models import SystemMessage, UserMessage

response = model.complete(
    messages=[
        UserMessage(content='HI'),
    ],
    model="Llama-3.3-70B-Instruct"
)

In [ ]:
print(response.choices[0].message.content)